<a href="https://colab.research.google.com/github/jylee2930/Basic_BIgDataAnalysis/blob/main/Teampreture(KNN).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# 1. 데이터 로드 및 전처리
def load_and_preprocess_data(file_path):
    """
    원주 기온 데이터 로드 및 전처리
    """
    # CSV 파일 읽기
    df = pd.read_csv('/content/wonju_temp.csv')

    # 날짜 컬럼에서 탭 문자 제거 및 날짜 형식 변환
    df['date'] = df['date'].str.replace('\t', '').str.strip()

    # 빈 행 제거
    df = df.dropna(subset=['date', 'mean_tmp', 'min_tmp', 'max_tmp'])

    # 날짜가 빈 문자열인 경우 제거
    df = df[df['date'] != '']

    # 날짜 형식 변환
    df['date'] = pd.to_datetime(df['date'])

    # 날짜순 정렬
    df = df.sort_values('date').reset_index(drop=True)

    print(f"데이터 로드 완료: {len(df)}일의 기온 데이터")
    print(f"데이터 기간: {df['date'].min().strftime('%Y-%m-%d')} ~ {df['date'].max().strftime('%Y-%m-%d')}")
    print(f"평균 기온 범위: {df['mean_tmp'].min():.1f}°C ~ {df['mean_tmp'].max():.1f}°C")

    return df

def create_features(df):
    """
    시계열 특성 및 기상 특성 생성
    """
    df = df.copy()

    # 날짜 관련 특성
    df['year'] = df['date'].dt.year
    df['month'] = df['date'].dt.month
    df['day'] = df['date'].dt.day
    df['day_of_year'] = df['date'].dt.dayofyear
    df['day_of_week'] = df['date'].dt.dayofweek
    df['quarter'] = df['date'].dt.quarter

    # 계절성 특성 (삼각함수로 순환적 특성 표현)
    df['month_sin'] = np.sin(2 * np.pi * df['month'] / 12)
    df['month_cos'] = np.cos(2 * np.pi * df['month'] / 12)
    df['day_of_year_sin'] = np.sin(2 * np.pi * df['day_of_year'] / 365)
    df['day_of_year_cos'] = np.cos(2 * np.pi * df['day_of_year'] / 365)
    df['day_of_week_sin'] = np.sin(2 * np.pi * df['day_of_week'] / 7)
    df['day_of_week_cos'] = np.cos(2 * np.pi * df['day_of_week'] / 7)

    # 기온 범위 특성
    df['temp_range'] = df['max_tmp'] - df['min_tmp']
    df['temp_mid'] = (df['max_tmp'] + df['min_tmp']) / 2

    # 과거 기온 정보 (lag features)
    for lag in [1, 2, 3, 7, 14, 30]:
        df[f'mean_tmp_lag_{lag}'] = df['mean_tmp'].shift(lag)
        df[f'min_tmp_lag_{lag}'] = df['min_tmp'].shift(lag)
        df[f'max_tmp_lag_{lag}'] = df['max_tmp'].shift(lag)
        df[f'temp_range_lag_{lag}'] = df['temp_range'].shift(lag)

    # 이동평균 특성
    for window in [3, 7, 14, 30]:
        df[f'mean_tmp_ma_{window}'] = df['mean_tmp'].rolling(window=window, min_periods=1).mean()
        df[f'min_tmp_ma_{window}'] = df['min_tmp'].rolling(window=window, min_periods=1).mean()
        df[f'max_tmp_ma_{window}'] = df['max_tmp'].rolling(window=window, min_periods=1).mean()
        df[f'temp_range_ma_{window}'] = df['temp_range'].rolling(window=window, min_periods=1).mean()

    # 이동표준편차 특성
    for window in [7, 14, 30]:
        df[f'mean_tmp_std_{window}'] = df['mean_tmp'].rolling(window=window, min_periods=1).std()
        df[f'temp_range_std_{window}'] = df['temp_range'].rolling(window=window, min_periods=1).std()

    # 기온 변화율 특성
    df['mean_tmp_diff_1'] = df['mean_tmp'].diff(1)
    df['mean_tmp_diff_7'] = df['mean_tmp'].diff(7)
    df['temp_range_diff_1'] = df['temp_range'].diff(1)

    # 계절별 평균과의 차이
    seasonal_mean = df.groupby(['month'])['mean_tmp'].transform('mean')
    df['temp_seasonal_anomaly'] = df['mean_tmp'] - seasonal_mean

    # 연도별 추세
    yearly_mean = df.groupby(['year'])['mean_tmp'].transform('mean')
    df['temp_yearly_anomaly'] = df['mean_tmp'] - yearly_mean

    # 결측값 처리 (앞뒤 값의 평균으로 채움)
    numeric_cols = df.select_dtypes(include=[np.number]).columns
    df[numeric_cols] = df[numeric_cols].fillna(method='bfill').fillna(method='ffill')

    print(f"특성 생성 완료: 총 {len(df.columns)}개 특성")

    return df

def prepare_ml_data(df, target_col='mean_tmp'):
    """등으
    머신러닝을 위한 데이터 준비
    """
    # 예측에 사용할 특성 선택 (날짜, 타겟 변수, area 제외)
    exclude_cols = ['date', 'area', target_col]
    feature_cols = [col for col in df.columns if col not in exclude_cols]

    X = df[feature_cols]
    y = df[target_col]

    # 초기 30일은 lag 특성 때문에 품질이 낮을 수 있으므로 제거
    X = X.iloc[30:]
    y = y.iloc[30:]

    print(f"ML 데이터 준비 완료: {len(X)}개 샘플, {len(feature_cols)}개 특성")
    print(f"특성 목록 (일부): {feature_cols[:10]}...")

    return X, y, feature_cols

def find_optimal_k(X, y, k_range=range(1, 31), cv_folds=5):
    """
    교차 검증을 통해 최적의 K 값 찾기
    """
    print("최적의 K 값을 찾는 중...")

    # 데이터 정규화
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    # 다양한 가중치 방법 테스트
    param_grid = {
        'n_neighbors': k_range,
        'weights': ['uniform', 'distance'],
        'metric': ['euclidean', 'manhattan']
    }

    knn = KNeighborsRegressor()
    grid_search = GridSearchCV(
        knn, param_grid, cv=cv_folds,
        scoring='neg_mean_squared_error',
        n_jobs=-1, verbose=1
    )

    grid_search.fit(X_scaled, y)

    best_params = grid_search.best_params_
    best_score = np.sqrt(-grid_search.best_score_)

    print(f"최적 파라미터: {best_params}")
    print(f"최적 RMSE (CV): {best_score:.3f}°C")

    # K 값에 따른 성능 시각화
    k_scores = []
    for k in k_range:
        knn_temp = KNeighborsRegressor(n_neighbors=k, weights='distance')
        scores = cross_val_score(knn_temp, X_scaled, y, cv=cv_folds, scoring='neg_mean_squared_error')
        k_scores.append(np.sqrt(-scores.mean()))

    plt.figure(figsize=(12, 5))

    plt.subplot(1, 2, 1)
    plt.plot(k_range, k_scores, 'bo-', linewidth=2, markersize=6)
    plt.xlabel('K-value')
    plt.ylabel('RMSE (Cross Validation)')
    plt.title('Model Performance by K Value')
    plt.grid(True, alpha=0.3)
    optimal_k = best_params['n_neighbors']
    plt.axvline(x=optimal_k, color='r', linestyle='--', alpha=0.8, label=f'Optimize K={optimal_k}')
    plt.legend()

    # 검증 곡선 (overfitting 확인)
    from sklearn.model_selection import validation_curve
    train_scores, val_scores = validation_curve(
        KNeighborsRegressor(weights='distance'), X_scaled, y,
        param_name='n_neighbors', param_range=k_range,
        cv=cv_folds, scoring='neg_mean_squared_error'
    )

    train_rmse = np.sqrt(-train_scores.mean(axis=1))
    val_rmse = np.sqrt(-val_scores.mean(axis=1))

    plt.subplot(1, 2, 2)
    plt.plot(k_range, train_rmse, 'o-', label='Train RMSE', alpha=0.8)
    plt.plot(k_range, val_rmse, 's-', label='Val  RMSE', alpha=0.8)
    plt.xlabel('K Value')
    plt.ylabel('RMSE')
    plt.title('Train vs Validation')
    plt.legend()
    plt.grid(True, alpha=0.3)

    plt.tight_layout()
    plt.show()

    return best_params

def train_knn_model(X, y, best_params, test_size=0.2):
    """
    최적 파라미터로 KNN 모델 학습
    """
    # 시계열 데이터이므로 시간 순서를 유지하여 분할
    split_index = int(len(X) * (1 - test_size))
    X_train, X_test = X.iloc[:split_index], X.iloc[split_index:]
    y_train, y_test = y.iloc[:split_index], y.iloc[split_index:]

    print(f"Trian_data: {len(X_train)}개")
    print(f"Test_data: {len(X_test)}개")

    # 데이터 정규화
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # 최적 파라미터로 모델 학습
    knn = KNeighborsRegressor(**best_params)
    knn.fit(X_train_scaled, y_train)

    # 예측
    y_pred_train = knn.predict(X_train_scaled)
    y_pred_test = knn.predict(X_test_scaled)

    # 성능 평가
    train_rmse = np.sqrt(mean_squared_error(y_train, y_pred_train))
    test_rmse = np.sqrt(mean_squared_error(y_test, y_pred_test))
    train_mae = mean_absolute_error(y_train, y_pred_train)
    test_mae = mean_absolute_error(y_test, y_pred_test)
    train_r2 = r2_score(y_train, y_pred_train)
    test_r2 = r2_score(y_test, y_pred_test)

    print("\n=== 모델 성능 평가 ===")
    print(f"훈련 데이터 - RMSE: {train_rmse:.3f}°C, MAE: {train_mae:.3f}°C, R²: {train_r2:.3f}")
    print(f"테스트 데이터 - RMSE: {test_rmse:.3f}°C, MAE: {test_mae:.3f}°C, R²: {test_r2:.3f}")

    return (knn, scaler, X_train, X_test, y_train, y_test,
            y_pred_train, y_pred_test, split_index)

def visualize_results(df, y_train, y_test, y_pred_train, y_pred_test, split_index):
    """
    예측 결과 시각화
    """
    fig, axes = plt.subplots(2, 3, figsize=(18, 12))

    # 1. 실제 vs 예측 (훈련 데이터)
    axes[0, 0].scatter(y_train, y_pred_train, alpha=0.6, s=20)
    min_val, max_val = y_train.min(), y_train.max()
    axes[0, 0].plot([min_val, max_val], [min_val, max_val], 'r--', lw=2)
    axes[0, 0].set_xlabel('Real Mean Temp (°C)')
    axes[0, 0].set_ylabel('Predict Mean Temp (°C)')
    axes[0, 0].set_title('1. Train: Real vs Predict')
    axes[0, 0].grid(True, alpha=0.3)

    # 2. 실제 vs 예측 (테스트 데이터)
    axes[0, 1].scatter(y_test, y_pred_test, alpha=0.6, s=20, color='orange')
    min_val, max_val = y_test.min(), y_test.max()
    axes[0, 1].plot([min_val, max_val], [min_val, max_val], 'r--', lw=2)
    axes[0, 1].set_xlabel('Real Mean Temp (°C)')
    axes[0, 1].set_ylabel('Predict Mean Temp  (°C)')
    axes[0, 1].set_title('2. Test: Real vs Predict')
    axes[0, 1].grid(True, alpha=0.3)

    # 3. 시계열 예측 결과
    train_dates = df['date'].iloc[30:30+split_index]
    test_dates = df['date'].iloc[30+split_index:]

    axes[0, 2].plot(train_dates, y_train, label='Trian', alpha=0.7, linewidth=1)
    axes[0, 2].plot(test_dates, y_test, label='Test', color='orange', linewidth=2)
    axes[0, 2].plot(test_dates, y_pred_test, label='Predict', color='red',
                   linestyle='--', linewidth=2)
    axes[0, 2].set_xlabel('Date')
    axes[0, 2].set_ylabel('Mean_temp(°C)')
    axes[0, 2].set_title('3. Time Serial Predict')
    axes[0, 2].legend()
    axes[0, 2].grid(True, alpha=0.3)

    # 4. 잔차 분석
    residuals_test = y_test - y_pred_test
    axes[1, 0].scatter(y_pred_test, residuals_test, alpha=0.6, s=20, color='green')
    axes[1, 0].axhline(y=0, color='r', linestyle='--', lw=2)
    axes[1, 0].set_xlabel('Predict_mean_temp (°C)')
    axes[1, 0].set_ylabel('Residual(real-predict)')
    axes[1, 0].set_title('4, Residual Analysis')
    axes[1, 0].grid(True, alpha=0.3)

    # 5. 잔차 히스토그램
    axes[1, 1].hist(residuals_test, bins=30, alpha=0.7, color='skyblue', edgecolor='black')
    axes[1, 1].set_xlabel('Residuals (°C)')
    axes[1, 1].set_ylabel('Frequency')
    axes[1, 1].set_title('5. Residual Distribution')
    axes[1, 1].axvline(x=0, color='r', linestyle='--', lw=2)
    axes[1, 1].grid(True, alpha=0.3)

    # 6. 월별 성능 분석
    test_dates_series = pd.Series(test_dates.values)
    monthly_rmse = []
    months = []

    y_test_reset = y_test.reset_index(drop=True)
    y_pred_test_series = pd.Series(y_pred_test)

    for month in range(1, 13):
        month_mask = test_dates_series.dt.month == month
        if month_mask.sum() > 0:
            month_y_test = y_test_reset[month_mask]
            month_y_pred = y_pred_test_series[month_mask]
            month_rmse = np.sqrt(mean_squared_error(month_y_test, month_y_pred))
            monthly_rmse.append(month_rmse)
            months.append(month)

    axes[1, 2].bar(months, monthly_rmse, alpha=0.7, color='lightcoral')
    axes[1, 2].set_xlabel('Month')
    axes[1, 2].set_ylabel('RMSE (°C)')
    axes[1, 2].set_title('Monthly Prediction Performance')
    axes[1, 2].set_xticks(months)
    axes[1, 2].grid(True, alpha=0.3)

    plt.tight_layout()
    plt.show()

    # 성능 요약 출력
    print("\n=== 상세 성능 분석 ===")
    print(f"잔차 평균: {residuals_test.mean():.3f}°C")
    print(f"잔차 표준편차: {residuals_test.std():.3f}°C")
    print(f"절대 오차 중앙값: {np.median(np.abs(residuals_test)):.3f}°C")
    print(f"95% 예측 구간: ±{np.percentile(np.abs(residuals_test), 95):.3f}°C")

def predict_future_temperature(model, scaler, df, feature_cols, days_ahead=30):
    """
    미래 기온 예측 (Autoregressive)
    """
    print(f"\n미래 {days_ahead}일간 기온 예측 중...")

    # 원본 데이터프레임의 기본 컬럼만 사용하여 예측을 위한 데이터프레임 생성
    base_cols = ['date', 'mean_tmp', 'min_tmp', 'max_tmp', 'area']
    future_df = df[base_cols].copy()
    predictions = []

    for i in range(days_ahead):
        # 1. 현재까지의 데이터를 기반으로 특성 생성
        df_with_features = create_features(future_df)

        # 2. 마지막 데이터 포인트를 사용하여 예측
        features_for_prediction = df_with_features[feature_cols].iloc[-1:]

        # 3. 스케일링 및 예측
        scaled_features = scaler.transform(features_for_prediction)
        predicted_temp = model.predict(scaled_features)[0]
        predictions.append(predicted_temp)

        # 4. 다음 날짜에 대한 새로운 행 생성
        last_date = future_df['date'].iloc[-1]
        next_date = last_date + timedelta(days=1)

        new_row = {
            'date': next_date,
            'mean_tmp': predicted_temp,
            'min_tmp': predicted_temp - 5,  # 단순 추정
            'max_tmp': predicted_temp + 5,  # 단순 추정
            'area': future_df['area'].iloc[-1]
        }

        # 5. 예측된 행을 데이터프레임에 추가하여 다음 반복에 사용
        future_df = pd.concat([future_df, pd.DataFrame([new_row])], ignore_index=True)

        if (i + 1) % 7 == 0:
            print(f"{i+1}일 후 ({next_date.strftime('%Y-%m-%d')}): {predicted_temp:.1f}°C")

    # 예측 결과 시각화
    prediction_dates = future_df['date'].iloc[-days_ahead:]

    plt.figure(figsize=(15, 8))

    # 최근 60일 실제 데이터
    recent_data = df.tail(60)
    plt.plot(recent_data['date'], recent_data['mean_tmp'],
             'b-', label='Real Temp', linewidth=2, alpha=0.8)

    # 미래 예측 데이터
    plt.plot(prediction_dates, predictions,
             'r--', label='Predict Temp', linewidth=2, marker='o', markersize=4)

    plt.xlabel('Date')
    plt.ylabel('Mean Temp (°C)')
    plt.title(f'Predcit Temp of Wonju (Future {days_ahead}day)')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

    print(f"\n예측 요약:")
    print(f"Predict Mean Temp: {np.mean(predictions):.1f}°C")
    print(f"Predict Max Temp: {np.max(predictions):.1f}°C")
    print(f"Predict Min Temp: {np.min(predictions):.1f}°C")

    return predictions, prediction_dates

def analyze_feature_importance(model, scaler, X, y, feature_cols, top_n=20):
    """
    특성 중요도 분석 (Permutation Importance)
    """
    from sklearn.inspection import permutation_importance

    print("특성 중요도 분석 중...")

    X_scaled = scaler.transform(X)

    # Permutation importance 계산
    perm_importance = permutation_importance(
        model, X_scaled, y, n_repeats=10, random_state=42, scoring='neg_mean_squared_error'
    )

    # 중요도 정리
    importance_df = pd.DataFrame({
        'feature': feature_cols,
        'importance': perm_importance.importances_mean,
        'std': perm_importance.importances_std
    }).sort_values('importance', ascending=False)

    # 상위 특성들 시각화
    plt.figure(figsize=(12, 8))
    top_features = importance_df.head(top_n)

    plt.barh(range(len(top_features)), top_features['importance'],
             xerr=top_features['std'], alpha=0.7, color='skyblue')
    plt.yticks(range(len(top_features)), top_features['feature'])
    plt.xlabel('Permutation Importance')
    plt.title(f'Top {top_n} Importance Feature')
    plt.gca().invert_yaxis()
    plt.tight_layout()
    plt.show()

    print(f"\nTop {min(10, len(importance_df))}Importance Feature:")
    for i, (_, row) in enumerate(importance_df.head(10).iterrows()):
        print(f"{i+1:2d}. {row['feature']:25s}: {row['importance']:.4f} ± {row['std']:.4f}")

    return importance_df

In [ ]:
# 메인 실행 함수

"""
메인 실행 함수
"""
print("=== 원주 기온 데이터 KNN 예측 시스템 ===\n")

# 1. 데이터 로드 및 전처리
df = load_and_preprocess_data('/content/wonju_temp.csv')

# 2. 특성 생성
df_with_features = create_features(df)

# 3. 머신러닝 데이터 준비
X, y, feature_cols = prepare_ml_data(df_with_features, target_col='mean_tmp')

# 4. 최적 하이퍼파라미터 탐색
best_params = find_optimal_k(X, y, k_range=range(3, 21))

# 5. 모델 학습
(model, scaler, X_train, X_test, y_train, y_test,
    y_pred_train, y_pred_test, split_index) = train_knn_model(X, y, best_params)

# 6. 결과 시각화
visualize_results(df_with_features, y_train, y_test, y_pred_train, y_pred_test, split_index)

# 7. 특성 중요도 분석
importance_df = analyze_feature_importance(model, scaler, X_test, y_test, feature_cols)

# 8. 미래 예측
predictions, dates = predict_future_temperature(
    model, scaler, df_with_features, feature_cols, days_ahead=30
)



